<a href="https://www.rhyme.com"> <img src="https://www.rhyme.com/assets/img/logo-dark.png" alt="Header" style="width: 100px;"/> </a>
<h1 align=center> Image Super Resolution using Autoencoders</h1>

<img src="images/high_res_v_low_res.jpg" width=550px>

## Task 1: Project Overview and Import Libraries

In [1]:
import os
import re
from scipy import ndimage, misc
from skimage.transform import resize, rescale
from matplotlib import pyplot
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(0)

from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Conv2DTranspose, UpSampling2D, add
from tensorflow.keras.models import Model
from tensorflow.keras import regularizers
import tensorflow as tf
print(tf.__version__)

2.1.0


## Task 2: What are Autoencoders?

<img src="images/autoencoder.jpg">
Credit: Autoencoder Schema by <a href="https://blog.keras.io/img/ae/autoencoder_schema.jpg">Francois Chollet, 2016</a>.

<h4 align=center>Encoder Architecture</h4>
<img src="images/encoder.png" width=450px align=center>

## Task 3: Build the Encoder

In [2]:
input_image = Input(shape=(256,256,3))
layer1 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_image)
layer2 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer1)
layer3 = MaxPooling2D(padding='same')(layer2)
layer4 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer3)
layer5 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer4)
layer6 = MaxPooling2D(padding='same')(layer5)
layer7 = Conv2D(256, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer6)
encoder = Model(input_image, layer7)

In [3]:
encoder.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 256, 256, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 256, 256, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 128, 128, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 128)       0     

## Task 4: Build the Decoder to Complete the Network

<img src="images/decoder.png" width=450px>

In [4]:
#input_image = Input(shape=(256,256,3))
#layer1 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(input_image)
#layer2 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer1)
#layer3 = MaxPooling2D(padding='same')(layer2)
#layer4 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer3)
#layer5 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer4)
#layer6 = MaxPooling2D(padding='same')(layer5)
#layer7 = Conv2D(256, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer6)

layer8 = UpSampling2D()(layer7)
layer9 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer8)
layer10 = Conv2D(128, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer9)
layer11 = add([layer5, layer10])
layer12 = UpSampling2D()(layer11)
layer13 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer12)
layer14 = Conv2D(64, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer13)
layer15 = add([layer14, layer2])

decoder = Conv2D(3, (3,3), padding='same', activation='relu', activity_regularizer=regularizers.l1(10e-10))(layer15)
autoencoder = Model(input_image, decoder)

In [5]:
autoencoder.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 64) 1792        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 256, 64) 36928       conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 128, 128, 64) 0           conv2d_1[0][0]                   
____________________________________________________________________________________________

In [6]:
autoencoder.compile(optimizer='adadelta', loss='mean_squared_error')

## Task 5: Create Dataset and Specify Training Routine

In [ ]:
def train_batches(just_load_dataset=False):

    batches = 256 

    batch = 0 
    batch_nb = 0 
    max_batches = -1 
    
    ep = 4 

    images = []
    x_train_n = []
    x_train_down = []
    
    x_train_n2 = [] 
    x_train_down2 = []
    
    for root, dirnames, filenames in os.walk("/home/rhyme/Desktop/Project/data/cars_train"):
        for filename in filenames:
            if re.search("\.(jpg|jpeg|JPEG|png|bmp|tiff)$", filename):
                if batch_nb == max_batches: 
                    return x_train_n2, x_train_down2
                filepath = os.path.join(root, filename)
                image = pyplot.imread(filepath)
                if len(image.shape) > 2:
                        
                    image_resized = resize(image, (256, 256))
                    x_train_n.append(image_resized)
                    x_train_down.append(rescale(rescale(image_resized, 0.5), 2.0))
                    batch += 1
                    if batch == batches:
                        batch_nb += 1

                        x_train_n2 = np.array(x_train_n)
                        x_train_down2 = np.array(x_train_down)
                        
                        if just_load_dataset:
                            return x_train_n2, x_train_down2
                        
                        print('Training batch', batch_nb, '(', batches, ')')

                        autoencoder.fit(x_train_down2, x_train_n2,
                            epochs=ep,
                            batch_size=10,
                            shuffle=True,
                            validation_split=0.15)
                    
                        x_train_n = []
                        x_train_down = []
                    
                        batch = 0

    return x_train_n2, x_train_down2

## Task 6: Load the Dataset and Pre-trained Model

## Task 7: Model Predictions and Visualizing the Results